In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from keras.applications.densenet import preprocess_input, DenseNet121

train_df = pd.read_csv('/kaggle/input/petfinder-adoption-prediction/train/train.csv')
img_size = 256
batch_size = 16

if np.version.version<"1.23":
    pass
else:
    ! pip install -q --force-reinstall numpy==1.22.4 # esto es porque densenet121 necesita una librería numpy menor a 1.23

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 57.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible

In [2]:
pet_ids = train_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

In [3]:
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

## Instanciación del modelo

In [4]:
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
inp = Input((256,256,3))
backbone = DenseNet121(input_tensor = inp, include_top = False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)

m = Model(inp,out)

29084464/29084464 [==============================] - 0s 0us/step


## Train set

In [5]:
features = {}
for b in tqdm(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("/kaggle/input/petfinder-adoption-prediction/train_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

  0%|          | 0/938 [00:00<?, ?it/s]

1/1 [==============================] - 0s 41ms/step


[ WARN:0@44.884] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/1bc0f89d8-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@44.885] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/1bf24d8be-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@44.924] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/7843a9dca-1.jpg'): can't open/read file: check file path/integrity


1/1 [==============================] - 0s 40ms/step


[ WARN:0@45.475] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/c740abfe1-1.jpg'): can't open/read file: check file path/integrity


...

1/1 [==============================] - 0s 33ms/step


[ WARN:0@340.875] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/d3faf6a0e-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@341.064] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/a2317e5b0-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@341.100] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/9ca247f84-1.jpg'): can't open/read file: check file path/integrity


1/1 [==============================] - 0s 34ms/step


[ WARN:0@341.458] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/train_images/e7f7066b6-1.jpg'): can't open/read file: check file path/integrity


1/1 [==============================] - 1s 973ms/step


In [6]:
train_feats = pd.DataFrame.from_dict(features, orient='index')

train_feats.to_parquet("train_img_features.parquet")
train_feats.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
86e1089a3,0.002000,0.167800,0.019715,0.015896,0.068162,0.002216,0.005042,0.004828,0.050760,0.047626,...,0.787698,0.176625,0.575706,1.088628,0.439557,0.520460,1.547071,0.832572,0.599094,0.763349
6296e909a,0.002858,0.107450,0.019916,0.023482,0.174765,0.002297,0.005031,0.006338,0.083378,0.049948,...,0.628259,0.686865,0.563999,0.968190,1.070276,1.545740,0.894411,0.838595,0.468236,0.916672
3422e4906,0.002734,0.072015,0.024455,0.018021,0.154207,0.001946,0.004211,0.001576,0.100046,0.039717,...,0.579116,0.557625,1.131406,0.720514,1.496672,0.870955,1.289683,1.184461,0.465114,0.892826
5842f1ff5,0.002106,0.274519,0.054815,0.013727,0.089969,0.001650,0.005506,0.004295,0.118727,0.034790,...,1.295853,0.326143,0.291669,1.608086,1.119175,1.470888,0.591445,0.832753,0.483021,1.134128
850a43f90,0.002185,0.174022,0.044818,0.016244,0.169775,0.002075,0.004421,0.004157,0.099671,0.034441,...,1.092662,0.669893,0.395784,0.886076,1.219730,1.033964,1.065686,0.304053,0.438069,0.676818


### Test set

In [7]:
# Cargo el test set
test_df = pd.read_csv('/kaggle/input/petfinder-adoption-prediction/test/test.csv')

# Guardo los IDs Y defino la cantidad de batches
pet_ids = test_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

In [8]:
features = {}
for b in tqdm(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("/kaggle/input/petfinder-adoption-prediction/test_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

  0%|          | 0/249 [00:00<?, ?it/s]

1/1 [==============================] - 0s 33ms/step


[ WARN:0@345.742] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/cbe2df167-1.jpg'): can't open/read file: check file path/integrity


1/1 [==============================] - 0s 37ms/step


[ WARN:0@349.835] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/637230592-1.jpg'): can't open/read file: check file path/integrity


...

1/1 [==============================] - 0s 32ms/step


[ WARN:0@422.918] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/19653e9f4-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@423.061] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/a848b0e4d-1.jpg'): can't open/read file: check file path/integrity


1/1 [==============================] - 0s 34ms/step


[ WARN:0@423.144] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/4fa94b6c0-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@423.387] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/0cdd2842f-1.jpg'): can't open/read file: check file path/integrity
[ WARN:0@423.508] global loadsave.cpp:248 findDecoder imread_('/kaggle/input/petfinder-adoption-prediction/test_images/acbd61638-1.jpg'): can't open/read file: check file path/integrity


1/1 [==============================] - 1s 1s/step


In [9]:
test_feats = pd.DataFrame.from_dict(features, orient='index')

test_feats.to_parquet("test_img_features.parquet")
test_feats.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
e2dfc2935,0.002276,0.098883,0.038320,0.021242,0.110261,0.002695,0.010345,0.005138,0.031219,0.035069,...,0.857912,0.187533,1.261031,1.180686,0.599100,0.522733,2.185492,0.647163,1.288971,0.573172
f153b465f,0.001814,0.147772,0.048576,0.012775,0.118605,0.002199,0.006744,0.005028,0.083216,0.041131,...,0.467064,0.377376,1.124959,1.781448,0.723128,1.430720,1.368843,1.211993,0.700757,0.778651
3c90f3f54,0.002446,0.104848,0.034093,0.017822,0.172918,0.002003,0.007518,0.003408,0.115374,0.054852,...,0.548644,0.184161,0.981034,0.761217,0.923164,1.352976,1.063603,0.754442,0.946966,0.542509
e02abc8a3,0.001774,0.130502,0.007370,0.013488,0.058525,0.002659,0.008706,0.005486,0.057609,0.025245,...,1.256507,0.478790,0.988276,2.068765,1.675209,2.998518,1.169877,0.274677,0.504366,1.143076
09f0df7d1,0.001910,0.112259,0.054931,0.013513,0.089616,0.002073,0.006150,0.004488,0.071213,0.036839,...,0.895282,0.257886,1.218698,1.412368,0.809647,0.801638,1.819161,1.295868,0.702705,0.610848
